In [ ]:
# ==================== ขั้นตอนที่ 1: ติดตั้ง Dependencies ====================
# (ใน Colab จะต้องรันคำสั่ง pip install ก่อน)

!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" huggingface_hub hf_transfer
!pip install --no-deps unsloth
!pip install trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 376.2/376.2 kB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.8/162.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.7.

In [ ]:
# ==================== ขั้นตอนที่ 2: Mount Google Drive ====================
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# ==================== ขั้นตอนที่ 3: Import Libraries ====================
import torch
print(f"PyTorch version: {torch.__version__}")

from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template, train_on_responses_only
from transformers.modeling_utils import PreTrainedModel
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments, DataCollatorForSeq2Seq
from unsloth import is_bfloat16_supported

PyTorch version: 2.6.0+cu124
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
# ==================== ขั้นตอนที่ 4: เตรียมข้อมูล Training ====================
# ข้อมูล FAQ สำหรับผลิตภัณฑ์ Damini
json_data = [
    {
        "conversation": [
            {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"},
            {"role": "assistant", "content": "This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "Who is the target audience for this product Damini"},
            {"role": "assistant", "content": "Data scientists, analysts, researchers and businesses looking to extract valuable insights from complex datasets."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the key features of the product Damini?"},
            {"role": "assistant", "content": "Data ingestion from diverse sources, data cleaning and pre-processing tools, advanced statistical analysis, predictive modeling, visualization capabilities, and customizable dashboards."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What type of data formats does the product Damini support?"},
            {"role": "assistant", "content": "CSV, Excel, JSON, XML, SQL databases, and various API integrations."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the different deployment options available for Damini?"},
            {"role": "assistant", "content": "Cloud-based subscription model with access through a web interface, on-premises deployment for high security needs."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the system requirements for using the product Damini?"},
            {"role": "assistant", "content": "Minimum system specifications including required operating systems, memory, and network bandwidth are specified on the product site. Please check them."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "How is the product Damini priced?"},
            {"role": "assistant", "content": "Based on a tiered subscription model with varying features and data storage capacity."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the payment options for Damini?"},
            {"role": "assistant", "content": "Credit card, bank transfer, purchase orders for enterprise clients."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "Is there a free trial available for this product Damini?"},
            {"role": "assistant", "content": "Yes, a limited-time free trial with access to core functionalities is available for new users."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "How can I access customer support for product Damini?"},
            {"role": "assistant", "content": "Through our online customer support portal, email, or phone during the business hours."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What types of support are offered for Damini?"},
            {"role": "assistant", "content": "User Documentations, tutorials, technical troubleshooting and personalized assistance from our dedicated support team."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "How does the product Damini protect my data?"},
            {"role": "assistant", "content": "Industry-standard encryption protocols, strict data privacy policies, and compliance with relevant data protection regulations."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the data residency options for Damini?"},
            {"role": "assistant", "content": "Ability to store data in specific geographic locations based on customer requirements."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "What are the limitations of the product Damini?"},
            {"role": "assistant", "content": "We have mentioned any specific data size restrictions, complex modeling limitations, or features not yet available etc. on the product site. Please check there."}
        ]
    },
    {
        "conversation": [
            {"role": "user", "content": "How can I provide feedback on the product Damini?"},
            {"role": "assistant", "content": "Through the feedback section on the user interface, dedicated feedback channels, or direct contact with the support team."}
        ]
    },
]

In [ ]:
# แปลงข้อมูลเป็น Dataset
dataset = Dataset.from_list(json_data)
print(f"Dataset size: {len(dataset)}")

Dataset size: 15


In [ ]:
# ==================== ขั้นตอนที่ 5: โหลดและเตรียม Model ====================
# กำหนด configuration
max_seq_length = 2048
dtype = None  # เลือกอัตโนมัติ (float16, bfloat16, etc.)
load_in_4bit = True  # ใช้ 4-bit quantization

# โหลด base model
print("Loading base model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

# ตั้งค่า chat template
tokenizer = get_chat_template(
    tokenizer,
    chat_template="llama-3.2",
)
tokenizer.pad_token = tokenizer.eos_token

Loading base model...
==((====))==  Unsloth 2025.7.3: Fast Llama patching. Transformers: 4.53.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [ ]:
# ==================== ขั้นตอนที่ 6: ทดสอบ Model ก่อน Fine-tuning ====================
print("\n=== ทดสอบ Model ก่อน Fine-tuning ===")
FastLanguageModel.for_inference(model)


=== ทดสอบ Model ก่อน Fine-tuning ===


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072, padding_idx=128004)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=3072, out_features=3072, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm

In [ ]:
# ทดสอบการตอบคำถาม
test_messages = [
    {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"}
]

inputs = tokenizer.apply_chat_template(
    test_messages,
    tokenize=True,
    add_generation_prompt=True,
    return_tensors="pt",
    padding=True,
).to("cuda")

attention_mask = inputs != tokenizer.pad_token_id

outputs = model.generate(
    input_ids=inputs,
    attention_mask=attention_mask,
    max_new_tokens=64,
    use_cache=True,
    temperature=0.6,
    min_p=0.1,
)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Response before fine-tuning: {text}")

Response before fine-tuning: system

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

user

What is the primary function of the product Damini of AtoZ company?assistant

I couldn't find any information on a company called AtoZ or a product called Damini. It's possible that it's a private or local company, or it may not have a significant online presence. Can you provide more context or details about AtoZ and Damini?


In [ ]:
# ================ ขั้นตอนที่ 7: เตรียม Model สำหรับ Fine-tuning ====================
print("\n=== เตรียม Model สำหรับ Fine-tuning ===")

# เพิ่ม LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj", "linear",
    ],
)


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.1.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.



=== เตรียม Model สำหรับ Fine-tuning ===
Unsloth: You added custom modules, but Unsloth hasn't optimized for this.
Beware - your finetuning might be noticeably slower!


Unsloth 2025.7.3 patched 28 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [ ]:
# ==================== ขั้นตอนที่ 8: Format Dataset ====================
print("\n=== Format Dataset ===")

def formatting_prompts_func(examples):
    """แปลง conversation เป็นรูปแบบที่ model เข้าใจ"""
    convos = examples["conversation"]
    texts = [
        tokenizer.apply_chat_template(
            convo,
            tokenize=False,
            add_generation_prompt=False
        ) for convo in convos
    ]
    return {"text": texts}

dataset = dataset.map(formatting_prompts_func, batched=True)

# แสดงตัวอย่าง formatted data
print("Original conversation:")
print(dataset[0]["conversation"])
print("\nFormatted text:")
print(dataset[0]["text"])



=== Format Dataset ===


Map:   0%|          | 0/15 [00:00<?, ? examples/s]

Original conversation:
[{'content': 'What is the primary function of the product Damini of AtoZ company?', 'role': 'user'}, {'content': 'This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms.', 'role': 'assistant'}]

Formatted text:
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is the primary function of the product Damini of AtoZ company?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

This product is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms.<|eot_id|>


In [ ]:
# ==================== ขั้นตอนที่ 9: ตั้งค่า Trainer ====================
print("\n=== ตั้งค่า Trainer ===")

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=15,
        learning_rate=2e-4,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",
    ),
)

# ตั้งค่าให้ train เฉพาะ assistant responses
trainer = train_on_responses_only(
    trainer,
    instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
    response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
)


=== ตั้งค่า Trainer ===


Unsloth: Tokenizing ["text"]:   0%|          | 0/15 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/15 [00:00<?, ? examples/s]

In [ ]:
# ==================== ขั้นตอนที่ 10: เริ่ม Training ====================
print("\n=== เริ่ม Training ===")
trainer_stats = trainer.train()


=== เริ่ม Training ===


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 15 | Num Epochs = 15 | Total steps = 30
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 24,313,856 of 3,237,063,680 (0.75% trained)


Step,Training Loss
1,3.323900
2,4.118700
3,3.810900
4,3.143900
5,2.980400
6,3.070500
7,2.557600
8,2.437700
9,1.840400
10,2.298400


In [ ]:
# ==================== ขั้นตอนที่ 11: บันทึก Model ====================
print("\n=== บันทึก Model ===")
my_model = "Llama-3.2-3B-Instruct-bnb-4bit-finetuned1"
model.save_pretrained(my_model)
tokenizer.save_pretrained(my_model)
print(f"Model saved to: {my_model}")


=== บันทึก Model ===
Model saved to: Llama-3.2-3B-Instruct-bnb-4bit-finetuned1


In [ ]:
# ================= ขั้นตอนที่ 12: ทดสอบ Model หลัง Fine-tuning ====================
print("\n=== ทดสอบ Model หลัง Fine-tuning ===")

def generate_text(prompt, max_length=100):
    """ฟังก์ชันสำหรับทดสอบ model"""
    inputs = tokenizer(prompt, return_tensors="pt")
    inputs = {k: v.to(model.device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_length=max_length,
            num_return_sequences=1,
            temperature=0.5,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generated_text


=== ทดสอบ Model หลัง Fine-tuning ===


In [ ]:
# ทดสอบ model
test_prompt = "What is the primary function of the product Damini of AtoZ company?"
result = generate_text(test_prompt)
print(f"Input: {test_prompt}")
print(f"Output: {result}")

print("\n=== การ Fine-tuning เสร็จสมบูรณ์ ===")

Input: What is the primary function of the product Damini of AtoZ company?
Output: What is the primary function of the product Damini of AtoZ company? Data ingestion, data cleaning, predictive analytics, machine learning-based models, visualization tools, etc.
The primary function of the product is to help users manage and analyze large datasets through advanced data processing capabilities, including data ingestion, data cleaning, predictive analytics, machine learning-based models, visualization tools, etc. The product is designed to provide a comprehensive platform for data analysis and modeling, enabling users to extract valuable insights from complex

=== การ Fine-tuning เสร็จสมบูรณ์ ===


# -*- coding: utf-8 -*-
"""
คำแนะนำเพิ่มเติมสำหรับ Fine-tuning LLaMA 3.2 3B
เพิ่มการจัดการข้อมูล, การประเมินผล, และการปรับปรุงประสิทธิภาพ
"""*italicized text*

In [ ]:
import json
import torch
import numpy as np
from datasets import Dataset
from sklearn.model_selection import train_test_split
from transformers import TrainingArguments, DataCollatorForSeq2Seq, EarlyStoppingCallback
from trl import SFTTrainer
from unsloth import FastLanguageModel, is_bfloat16_supported
from unsloth.chat_templates import get_chat_template, train_on_responses_only
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import wandb  # สำหรับ logging (ถ้าต้องการ)


In [ ]:
# ==================== 1. ปรับปรุงการจัดการข้อมูล ====================

def create_enhanced_dataset():
    """สร้างชุดข้อมูลที่มีความหลากหลายมากขึ้น"""

    # ข้อมูล FAQ พื้นฐาน
    base_data = [
        {
            "conversation": [
                {"role": "user", "content": "What is the primary function of the product Damini of AtoZ company?"},
                {"role": "assistant", "content": "Damini is a cloud-based software solution that allows users to manage and analyze large datasets through advanced machine learning algorithms. It provides comprehensive data analytics capabilities for businesses and researchers."}
            ]
        },
        # ... (ข้อมูลเดิม)
    ]

    # เพิ่มข้อมูล variations สำหรับคำถามเดียวกัน
    variations_data = [
        {
            "conversation": [
                {"role": "user", "content": "Can you tell me about Damini's main features?"},
                {"role": "assistant", "content": "Damini offers data ingestion from diverse sources, data cleaning and pre-processing tools, advanced statistical analysis, predictive modeling, visualization capabilities, and customizable dashboards."}
            ]
        },
        {
            "conversation": [
                {"role": "user", "content": "What does Damini do exactly?"},
                {"role": "assistant", "content": "Damini is designed to help users manage and analyze large datasets using machine learning algorithms. It's a cloud-based platform that provides comprehensive data analytics solutions."}
            ]
        },
        # เพิ่มคำถามในลักษณะต่างๆ
    ]

    # เพิ่มข้อมูล negative examples (คำถามที่ไม่เกี่ยวข้อง)
    negative_examples = [
        {
            "conversation": [
                {"role": "user", "content": "What's the weather like today?"},
                {"role": "assistant", "content": "I'm sorry, I can only provide information about Damini product from AtoZ company. For weather information, please check a weather service."}
            ]
        },
        {
            "conversation": [
                {"role": "user", "content": "Can you help me with cooking recipes?"},
                {"role": "assistant", "content": "I specialize in providing information about Damini product. For cooking recipes, I'd recommend consulting a cooking website or app."}
            ]
        },
    ]

    # รวมข้อมูลทั้งหมด
    all_data = base_data + variations_data + negative_examples

    return all_data


In [ ]:
# ==================== 2. การแบ่งข้อมูล Train/Validation ====================

def split_dataset(data, test_size=0.2, random_state=42):
    """แบ่งข้อมูลเป็น training และ validation sets"""

    train_data, val_data = train_test_split(
        data,
        test_size=test_size,
        random_state=random_state,
        stratify=None  # ถ้ามี categories สามารถใช้ stratify ได้
    )

    train_dataset = Dataset.from_list(train_data)
    val_dataset = Dataset.from_list(val_data)

    return train_dataset, val_dataset


In [ ]:
# ==================== 3. ฟังก์ชันการประเมินผล ====================

def evaluate_model(model, tokenizer, test_questions, expected_keywords):
    """ประเมินผลการทำงานของ model"""

    FastLanguageModel.for_inference(model)
    results = []

    for i, question in enumerate(test_questions):
        # สร้าง prompt
        messages = [{"role": "user", "content": question}]

        # Generate response
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,
        ).to("cuda")

        attention_mask = inputs != tokenizer.pad_token_id

        outputs = model.generate(
            input_ids=inputs,
            attention_mask=attention_mask,
            max_new_tokens=100,
            temperature=0.3,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

        response = tokenizer.decode(outputs[0], skip_special_tokens=True)

        # ตรวจสอบว่ามี keywords ที่คาดหวังหรือไม่
        keyword_found = any(keyword.lower() in response.lower() for keyword in expected_keywords[i])

        results.append({
            "question": question,
            "response": response,
            "keyword_found": keyword_found
        })

    return results

In [ ]:
# ==================== 4. การตั้งค่า Hyperparameters ที่ปรับปรุง ====================

def get_optimized_training_args(output_dir="outputs"):
    """ได้ training arguments ที่ปรับปรุงแล้ว"""

    return TrainingArguments(
        # Basic settings
        output_dir=output_dir,
        num_train_epochs=20,  # เพิ่มจำนวน epochs
        per_device_train_batch_size=1,  # ลดลงเพื่อประหยัด memory
        per_device_eval_batch_size=1,
        gradient_accumulation_steps=8,  # เพิ่มเพื่อชดเชย batch size ที่เล็ก

        # Learning rate settings
        learning_rate=1e-4,  # ลดลงเล็กน้อย
        warmup_steps=10,
        lr_scheduler_type="cosine",  # ใช้ cosine schedule

        # Optimization
        optim="adamw_8bit",
        weight_decay=0.01,
        max_grad_norm=1.0,  # gradient clipping

        # Logging and evaluation
        logging_steps=5,
        eval_steps=50,
        save_steps=100,
        evaluation_strategy="steps",
        save_strategy="steps",

        # Early stopping
        load_best_model_at_end=True,
        metric_for_best_model="eval_loss",
        greater_is_better=False,

        # Memory optimization
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        dataloader_pin_memory=False,

        # Reproducibility
        seed=3407,
        data_seed=3407,

        # Reporting
        report_to="none",  # หรือ "wandb" ถ้าต้องการ
        run_name=f"damini_finetuning_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
    )


In [ ]:
# ==================== 5. การปรับปรุง LoRA Parameters ====================

def get_optimized_lora_config():
    """ได้ LoRA configuration ที่ปรับปรุงแล้ว"""

    return {
        "r": 32,  # เพิ่มจาก 16 เป็น 32
        "lora_alpha": 64,  # เพิ่มจาก 16 เป็น 64
        "lora_dropout": 0.05,  # ลดลงเล็กน้อย
        "target_modules": [
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
            "embed_tokens", "lm_head"  # เพิ่ม embedding และ head layers
        ],
        "task_type": "CAUSAL_LM",
        "bias": "none",
    }

In [ ]:
# ==================== 6. การสร้าง Custom Callback ====================

class MetricsCallback:
    """Custom callback สำหรับติดตาม metrics"""

    def __init__(self):
        self.training_losses = []
        self.eval_losses = []
        self.learning_rates = []

    def on_log(self, args, state, control, model=None, logs=None, **kwargs):
        if logs:
            if "train_loss" in logs:
                self.training_losses.append(logs["train_loss"])
            if "eval_loss" in logs:
                self.eval_losses.append(logs["eval_loss"])
            if "learning_rate" in logs:
                self.learning_rates.append(logs["learning_rate"])

    def plot_metrics(self):
        """Plot training metrics"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Training loss
        if self.training_losses:
            axes[0, 0].plot(self.training_losses)
            axes[0, 0].set_title('Training Loss')
            axes[0, 0].set_xlabel('Steps')
            axes[0, 0].set_ylabel('Loss')

        # Validation loss
        if self.eval_losses:
            axes[0, 1].plot(self.eval_losses)
            axes[0, 1].set_title('Validation Loss')
            axes[0, 1].set_xlabel('Steps')
            axes[0, 1].set_ylabel('Loss')

        # Learning rate
        if self.learning_rates:
            axes[1, 0].plot(self.learning_rates)
            axes[1, 0].set_title('Learning Rate')
            axes[1, 0].set_xlabel('Steps')
            axes[1, 0].set_ylabel('LR')

        plt.tight_layout()
        plt.savefig('training_metrics.png')
        plt.show()

In [ ]:
# ==================== 7. ฟังก์ชันการบันทึกและโหลด Model ====================

def save_model_with_metadata(model, tokenizer, save_path, training_args, metrics):
    """บันทึก model พร้อมกับ metadata"""

    # สร้าง directory
    os.makedirs(save_path, exist_ok=True)

    # บันทึก model และ tokenizer
    model.save_pretrained(save_path)
    tokenizer.save_pretrained(save_path)

    # บันทึก metadata
    metadata = {
        "training_args": training_args.to_dict(),
        "metrics": metrics,
        "timestamp": datetime.now().isoformat(),
        "model_type": "llama-3.2-3b-finetuned",
        "task": "damini_product_qa"
    }

    with open(os.path.join(save_path, "metadata.json"), "w") as f:
        json.dump(metadata, f, indent=2)

    print(f"Model and metadata saved to {save_path}")


In [ ]:
# ==================== 8. ฟังก์ชัน Main Training ====================

def main_training():
    """ฟังก์ชันหลักสำหรับการ training"""

    # 1. เตรียมข้อมูล
    print("Preparing dataset...")
    data = create_enhanced_dataset()
    train_dataset, val_dataset = split_dataset(data)

    # 2. โหลด model
    print("Loading model...")
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
        max_seq_length=2048,
        dtype=None,
        load_in_4bit=True,
    )

    # 3. ตั้งค่า tokenizer
    tokenizer = get_chat_template(tokenizer, chat_template="llama-3.2")
    tokenizer.pad_token = tokenizer.eos_token

    # 4. เพิ่ม LoRA
    lora_config = get_optimized_lora_config()
    model = FastLanguageModel.get_peft_model(model, **lora_config)

    # 5. Format dataset
    def formatting_prompts_func(examples):
        convos = examples["conversation"]
        texts = [tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=False) for convo in convos]
        return {"text": texts}

    train_dataset = train_dataset.map(formatting_prompts_func, batched=True)
    val_dataset = val_dataset.map(formatting_prompts_func, batched=True)


In [ ]:
# 6. เตรียม trainer
    training_args = get_optimized_training_args()
    metrics_callback = MetricsCallback()

    trainer = SFTTrainer(
        model=model,
        tokenizer=tokenizer,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        dataset_text_field="text",
        max_seq_length=2048,
        data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer),
        args=training_args,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

In [ ]:
# 7. ตั้งค่า train เฉพาะ responses
    trainer = train_on_responses_only(
        trainer,
        instruction_part="<|start_header_id|>user<|end_header_id|>\n\n",
        response_part="<|start_header_id|>assistant<|end_header_id|>\n\n",
    )


In [ ]:
# 8. เริ่ม training
    print("Starting training...")
    trainer_stats = trainer.train()


In [ ]:
 # 9. ประเมินผล
    print("Evaluating model...")
    test_questions = [
        "What is Damini?",
        "How much does Damini cost?",
        "What formats does Damini support?",
    ]
    expected_keywords = [
        ["cloud-based", "software", "data", "analytics"],
        ["subscription", "tiered", "pricing"],
        ["CSV", "Excel", "JSON", "XML"]
    ]

    evaluation_results = evaluate_model(model, tokenizer, test_questions, expected_keywords)


In [ ]:
# 10. บันทึก model
    save_model_with_metadata(
        model,
        tokenizer,
        "damini_finetuned_model",
        training_args,
        evaluation_results
    )

In [ ]:
 # 11. แสดงผล
    metrics_callback.plot_metrics()

    print("Training completed successfully!")
    return model, tokenizer, trainer_stats

# ==================== 9. คำแนะนำเพิ่มเติม ====================

"""
คำแนะนำการใช้งาน:

1. **ข้อมูล (Data)**:
   - เพิ่มข้อมูลให้มากขึ้น (อย่างน้อย 100-500 คู่ QA)
   - ใส่ variations ของคำถามเดียวกัน
   - เพิ่ม negative examples
   - ใช้ data augmentation techniques

2. **Model Parameters**:
   - ปรับ LoRA rank (r) ตาม complexity ของ task
   - ทดลอง learning rate ต่างๆ (1e-5 ถึง 1e-3)
   - ใช้ cosine scheduler สำหรับ learning rate

3. **Training**:
   - ใช้ early stopping เพื่อป้องกัน overfitting
   - Monitor validation loss
   - บันทึก checkpoints เป็นระยะ

4. **Evaluation**:
   - ใช้ metrics หลากหลาย (BLEU, ROUGE, perplexity)
   - ทดสอบกับ real-world scenarios
   - ใช้ human evaluation

5. **Deployment**:
   - ใช้ quantization เพื่อลดขนาด model
   - ทดสอบ latency และ throughput
   - ตั้งค่า safety filters

6. **Monitoring**:
   - ใช้ Weights & Biases สำหรับ experiment tracking
   - Monitor GPU utilization
   - ติดตาม model performance ในระยะยาว
"""


In [ ]:
# เรียกใช้ main function
if __name__ == "__main__":
    model, tokenizer, stats = main_training()